In [1]:
import numpy as np 
import pandas as pd
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.metrics import mean_squared_log_error


import multiprocessing

### Cargar los datos

In [2]:
df = pd.read_csv("monthly-milk-production-pounds.csv", index_col=0, parse_dates=[0])

### Variable para el modelos

In [3]:
# CONSTANTES 
LAG = 12  # lagged values

In [4]:
def getParametres(tuned_parameters,X_train,y_train):
    
    print("# Tuning hyper-parameters for")
    print()
    clf = GridSearchCV(SVR(), tuned_parameters,cv=5, scoring='neg_mean_squared_error',verbose=10,n_jobs=-1)
    clf.fit(X_train, y_train)
    
    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()  

### Recortar los datos 

In [5]:
def window(ds, lags):
    tmp_ind = ds.index.to_numpy()[lags:]
    for i in range(lags):
        tmp = ds.iloc[:, 0].shift(-(i+1))
        ds = pd.concat([ds, tmp], axis=1)
        
    ds = ds.dropna()
    ds.index = tmp_ind
        
    return ds

ndf = window(df, LAG)

train_data = ndf[:'1974-12-31'].to_numpy()
test_data  = ndf['1974-12-31':].to_numpy()


In [6]:
print('Observaciones: %d' % (len(ndf)))
print('Dataset de entrenamiento:',train_data.shape)
print('Dataset de prueba:', test_data.shape)

Observaciones: 156
Dataset de entrenamiento: (144, 13)
Dataset de prueba: (12, 13)


In [7]:
X_train, y_train = train_data[:, :-1], train_data[:, -1]
X_test, y_test = test_data[:, :-1], test_data[:, -1]

### Obtener los parametros

In [8]:
#Obtencion de parametros

tuned_parameters = {'kernel': ('rbf','poly'), 
                    'C': [25,100,50,75]}
                     
# Designate distributions to sample hyperparameters from 
getParametres(tuned_parameters,X_train,y_train)

# Tuning hyper-parameters for

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  6.6min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  8.4min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 11.8min
[Parallel(n_jobs=-1)]: Done  38 out of  40 | elapsed: 17.2min remaining:   54.3s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed: 27.7min finished


Best parameters set found on development set:

{'C': 100, 'kernel': 'linear'}

